In [1]:
from datetime import date, datetime
import pandas as pd
import subprocess 
from subprocess import Popen
import time
import io, contextlib
from zipfile import ZipFile
from databases import Bawm, Crm, PosicaoDm1,PosicaoAdministrador,Boletador,BDS
from emailer import Email, EmailLer
from funcoes_datas import FuncoesDatas
from filemanager import FileMgmt
from datetime import date, datetime, timedelta
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1
from zipfile import ZipFile
import os
import calendar
import numpy as np
import ambiente
from funcoes_datas import FuncoesDatas
from objetos import Fundo
from pretty_html_table import build_table

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Puxa as Classes do Data Base
bawm = Bawm()
datas =  FuncoesDatas()
hoje = datas.hoje()


In [3]:
#Função que retorna o patriminio
def patrimonio(x):
    pl= Fundo(nome_conta_crm=x).plfut_est
    return pl 

In [4]:
#Puxa da base PO_Carteira Bawm todos os FICs que possuem produtos diferentes de cotas
ativos_verificar = bawm.analise_ativos_fics(hoje)
ativos_verificar = ativos_verificar[(ativos_verificar['FinanceiroFinal']!=0)&(ativos_verificar['TipoProduto']!='FUNDO')&(ativos_verificar['TipoProduto']!='BOLSA')&(ativos_verificar['TipoProduto']!='COMP')]

In [5]:
#Retorna o patrimonio por linha e retorna todos os potenciais produtos que podem gerar desenquadramento
ativos_verificar['PL']= ativos_verificar['NomeContaCRM'].apply(lambda x: patrimonio(x))
ativos_verificar['%PL'] = ativos_verificar['FinanceiroFinal'] /  ativos_verificar['PL']*100
ativos_verificar = ativos_verificar[ativos_verificar['%PL']>0.5]
ativos_verificar = ativos_verificar[['NomeContaCRM','NomeProduto','%PL']]
ativos_verificar = ativos_verificar[ativos_verificar['NomeContaCRM'].str.contains('FIC')]


KeyboardInterrupt



In [37]:
#Envia e-mail à PM

ativos_verificar = build_table(ativos_verificar, 'blue_light')

subject='Favor verificar essas posições nos FICs'
to=['portfolio@jbfo.com','mariana.drumond@jbfo.com','flavio.rotter@jbfo.com', 'luiza.gomes@juliusbaer.com']
text = '''Prezados .</h3><br>
<br>
Peço retirar as posições abaixo dos FICs, isso pode gerar desenquadramento .</h3><br>
{}<br>


'''.format(ativos_verificar,)

email = Email(to = to , subject = subject, text= text,send = False)

In [59]:
posicao = PosicaoDm1()
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=False)[['NomeContaCRM','TipoProduto','NomeProduto']].drop_duplicates()
cadastro = bawm.po_cadastro()
cadastro = cadastro[['NomeContaCRM','TipoCobranca','CNPJ']]
posicao_fundos = pd.merge(left=posicao_fundos,right=cadastro,on='NomeContaCRM',how='left')
posicao_fundos = posicao_fundos[['NomeContaCRM','TipoProduto','NomeProduto','TipoCobranca','CNPJ']]
posicao_fundos = posicao_fundos[posicao_fundos['TipoCobranca']!='FundoExclusivo']
posicao_fundos = posicao_fundos.groupby(['NomeContaCRM','TipoProduto','CNPJ']).count().reset_index()[['NomeContaCRM','CNPJ','TipoProduto','NomeProduto']]
posicao_fundos = posicao_fundos[(posicao_fundos['TipoProduto']=='FUNDO')&(posicao_fundos['NomeProduto']<=3)]
posicao = posicao_fundos.reset_index(drop=True)
posicao.to_excel('fundosBoeso.xlsx')